In [17]:
import warnings
warnings.filterwarnings('ignore')

# 加载数据
import pandas as pd
import os

from fastai.tabular import *
df = pd.read_csv("train_set.csv")
df = df.rename(columns={'告警开始时间_timestamp':'notice_time', '告警标题':'title', '故障原因定位（大类）':'label', '涉及基站eNBID或小区ECGI':'netid',
                  '故障发生时间_timestamp':'fault_time'})
df.drop(['告警开始时间'],inplace=True,axis=1)
# df.drop(['fault_time'],inplace=True,axis=1)
df

,notice_time,title,label,fault_time,netid
0,1549296720,小区不可用告警,电力故障,1549296720,505346F7A6D536667EE8F73CD15C1179
1,1549297920,小区接收通道干扰噪声功率不平衡告警,电力故障,1549297920,597ACECFDC702C079940F608FA17BE76
2,1549298040,小区接收通道干扰噪声功率不平衡告警,电力故障,1549298040,F98EC4E70CBBD66A4005CC3EF9D8233D
3,1549298160,4G基站退服告警,电力故障,1549298160,8B52CD71E5397D8D818F641A56657863
4,1549298460,时钟参考源异常告警,硬件故障,1549298460,D28C3879A688B51BC7016E55131A3F2D
5,1549301040,小区不可用告警,硬件故障,1549301040,67E65EC1EE29F3971993424FD9DCEAA1
6,1549303740,射频单元驻波告警,传输故障,1549303740,476A1EBA3437F8117AD6C5D40C551FF7
7,1549304940,射频单元CPRI接口异常告警,电力故障,1549304940,B731EF07B1376AEE94C525C0ED8AEE88
8,1549305120,网元连接中断,硬件故障,1549305120,0C965544227EF071A4D6C247F77D0617
9,1549305600,网元连接中断,电力故障,1549305600,DB8AFB92757886A789D95AFB6D34300D


In [12]:
import os
import pandas as pd
import numpy as np
import time
from sklearn import preprocessing
from collections import Counter
from tqdm import *


train= pd.read_csv('训练告警.csv',encoding='gbk', engine='python')
test= pd.read_csv('测试告警.csv',encoding='gbk', engine='python')
train.columns=['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']
test.columns=['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']

train_label= pd.read_csv('训练故障工单.csv', encoding="gbk", engine='python')
test_label= pd.read_csv('测试故障工单.csv',encoding="gbk", engine='python')

#去掉空行
train=train[~train['涉及基站eNBID或小区ECGI'].isnull()].reset_index(drop=True)
train=train.drop_duplicates(['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)
train_label=train_label.drop_duplicates(['故障发生时间', '故障原因定位（大类）', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)
test=test.drop_duplicates(['告警开始时间', '告警标题', '涉及基站eNBID或小区ECGI']).reset_index(drop=True)

train['告警开始时间']=train['告警开始时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))
test['告警开始时间']=test['告警开始时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))

train_label['故障发生时间']=train_label['故障发生时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))
test_label['故障发生时间']=test_label['故障发生时间'].apply(lambda x:time.strptime(x,"%Y/%m/%d %H:%M"))

train['告警开始时间_timestamp']=train['告警开始时间'].apply(lambda x:int(time.mktime(x)))
test['告警开始时间_timestamp']=test['告警开始时间'].apply(lambda x:int(time.mktime(x)))

train_label['故障发生时间_timestamp']=train_label['故障发生时间'].apply(lambda x:int(time.mktime(x)))
test_label['故障发生时间_timestamp']=test_label['故障发生时间'].apply(lambda x:int(time.mktime(x)))

# 衍生告警是依赖于根源告警的，这里我们需要删除掉
train=train[ ~ train['告警标题'].str.contains('衍生告警')]
test=test[ ~ test['告警标题'].str.contains('衍生告警')] 

In [13]:
train_label.head()

,工单编号,故障发生时间,涉及基站eNBID或小区ECGI,故障原因定位（大类）,故障发生时间_timestamp
0,1,"(2019, 2, 4, 22, 26, 0, 0, 35, -1)",5F0A37BA3C8428C1B2ED83CEB714BA3F,电力故障,1549290360
1,2,"(2019, 2, 5, 0, 12, 0, 1, 36, -1)",505346F7A6D536667EE8F73CD15C1179,电力故障,1549296720
2,3,"(2019, 2, 5, 0, 32, 0, 1, 36, -1)",597ACECFDC702C079940F608FA17BE76,电力故障,1549297920
3,4,"(2019, 2, 5, 0, 34, 0, 1, 36, -1)",F98EC4E70CBBD66A4005CC3EF9D8233D,电力故障,1549298040
4,5,"(2019, 2, 5, 0, 36, 0, 1, 36, -1)",8B52CD71E5397D8D818F641A56657863,电力故障,1549298160


In [14]:
# 整合训练集和测试集
total_data = pd.concat([train,test],axis=0).reset_index(drop=True)
total_data = total_data.drop_duplicates().reset_index(drop=True)
total_data.head()

,告警开始时间,告警标题,涉及基站eNBID或小区ECGI,告警开始时间_timestamp
0,"(2019, 2, 5, 0, 1, 0, 1, 36, -1)",射频单元IR接口异常告警,383A218145EEA7892DA45BC4B2DFCBFB,1549296060
1,"(2019, 2, 5, 0, 1, 0, 1, 36, -1)",射频单元IR接口异常告警,2216E5B350E675E20B983E9D75F6B232,1549296060
2,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元交流掉电告警,3C8F2AA2E939E9608CA5FE2200E87D84,1549296120
3,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元业务不可用告警,3C8F2AA2E939E9608CA5FE2200E87D84,1549296120
4,"(2019, 2, 5, 0, 2, 0, 1, 36, -1)",射频单元光模块收发异常告警,713570937709ED63AFD495B85CD3B5B3,1549296120


In [24]:
title2id = {}
_id = 1
for x in set(total_data['告警标题']):
    title2id[x] = _id
    _id = _id +1
title2id

{'系统时钟失锁告警': 1,
 '网元遭受攻击告警': 2,
 'License Feature不可用告警': 3,
 '传输光模块故障告警': 4,
 '本地用户连续登录尝试失败告警': 5,
 '电调天线未校准告警': 6,
 'RHUB光模块故障告警': 7,
 '射频单元直流掉电告警': 8,
 '小区闭塞告警': 9,
 '射频单元光模块收发异常告警': 10,
 'RHUB CPRI接口异常告警': 11,
 '远程维护通道配置与运行数据不一致告警': 12,
 '4G基站退服告警': 13,
 '射频单元光模块/电接口不在位告警': 14,
 'BBU CPRI光模块故障告警': 15,
 '射频单元运行时拓扑异常告警': 16,
 'MAC错帧超限告警': 17,
 '时钟参考源异常告警': 18,
 '小区PCI冲突告警': 19,
 '单板输入电压异常告警': 20,
 'BBU IR光模块收发异常告警': 21,
 '单板硬件故障告警': 22,
 'BBU直流输出异常告警': 23,
 '未配置时钟参考源告警': 24,
 '电调天线马达故障告警': 25,
 'SCTP链路拥塞告警': 26,
 '射频单元软件运行异常告警': 27,
 '制式间机柜配置冲突告警': 28,
 'BBU IR光模块故障告警': 29,
 'SCTP链路故障告警': 30,
 '星卡天线故障告警': 31,
 'RHUB与pRRU间链路异常告警': 32,
 '射频单元维护链路异常告警': 33,
 '射频单元CPRI接口异常告警': 34,
 '射频单元时钟异常告警': 35,
 '单板温度异常告警': 36,
 'RHUB光模块/电接口不在位告警': 37,
 '星卡时钟输出异常告警': 38,
 '射频单元IR接口异常告警': 39,
 '电调天线数据丢失告警': 40,
 'BBU IR光模块/电接口不在位告警': 41,
 '电调天线运行数据异常告警': 42,
 'BBU IR接口异常告警': 43,
 '射频单元输入电源能力不足告警': 44,
 '传输光接口异常告警': 45,
 '系统时钟不可用告警': 46,
 '射频单元硬件故障告警': 47,
 'RHUB交流掉电告警': 48,
 'BBU CPRI光模块/电接口不在位告警': 49

In [28]:
new_data=pd.DataFrame(columns=["netid","label"])
for i,row in train_label.iterrows():
    m = df[(df['netid']==row['涉及基站eNBID或小区ECGI'])& \
            (df['fault_time'] == row['故障发生时间_timestamp'])].reset_index(drop=True)
    if m.shape[0] == 0:
        continue
    titles = set(m["title"])
    x_dict = {'netid': row['涉及基站eNBID或小区ECGI'], 'label': row['故障原因定位（大类）']}
    for title in title2id:
        if title in titles:
            x_dict["title_%s" % title2id[title]] = 1
        else:
            x_dict["title_%s" % title2id[title]] = 0
#     print(x_dict)
    new_data = new_data.append(x_dict, ignore_index=True)
new_data

,netid,label,title_1,title_10,title_11,title_12,title_13,title_14,title_15,title_16,...,title_70,title_71,title_72,title_73,title_74,title_75,title_76,title_77,title_8,title_9
0,505346F7A6D536667EE8F73CD15C1179,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,597ACECFDC702C079940F608FA17BE76,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F98EC4E70CBBD66A4005CC3EF9D8233D,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8B52CD71E5397D8D818F641A56657863,电力故障,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D28C3879A688B51BC7016E55131A3F2D,硬件故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,67E65EC1EE29F3971993424FD9DCEAA1,硬件故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,476A1EBA3437F8117AD6C5D40C551FF7,传输故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,B731EF07B1376AEE94C525C0ED8AEE88,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0C965544227EF071A4D6C247F77D0617,硬件故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,DB8AFB92757886A789D95AFB6D34300D,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
new_data.head()

,netid,label,title_1,title_10,title_11,title_12,title_13,title_14,title_15,title_16,...,title_70,title_71,title_72,title_73,title_74,title_75,title_76,title_77,title_8,title_9
0,505346F7A6D536667EE8F73CD15C1179,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,597ACECFDC702C079940F608FA17BE76,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,F98EC4E70CBBD66A4005CC3EF9D8233D,电力故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8B52CD71E5397D8D818F641A56657863,电力故障,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,D28C3879A688B51BC7016E55131A3F2D,硬件故障,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
dep_var = 'label' # y
cat_names = set(new_data.columns) - set(['label', 'netid'])
cont_names = []
procs = [FillMissing, Categorify, Normalize]

In [40]:
path = Path('.')
test = TabularList.from_df(new_data.iloc[15511:18511].copy(), path=path, cat_names=cat_names, cont_names=cont_names)
data = (TabularList.from_df(new_data, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(15511,18511)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [42]:
learn = tabular_learner(data, layers=[200,100],emb_szs={'native-country': 10}, metrics=[accuracy])

In [55]:
learn.fit(10, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.586451,0.569155,0.802667,00:06
1,0.591176,0.572184,0.799333,00:06
2,0.570090,0.565242,0.793000,00:06
3,0.587700,0.567590,0.804667,00:06
4,0.580773,0.568771,0.804667,00:06
5,0.588626,0.565501,0.802667,00:06
6,0.573034,0.566519,0.801667,00:06
7,0.581563,0.564365,0.800667,00:06
8,0.562208,0.568929,0.808667,00:06
9,0.587317,0.569138,0.802333,00:06


In [56]:
row = new_data.iloc[0]
row

netid       505346F7A6D536667EE8F73CD15C1179
label                                   电力故障
title_1                                    0
title_10                                   0
title_11                                   0
                          ...               
title_75                                   0
title_76                                   1
title_77                                   0
title_8                                    0
title_9                                    0
Name: 0, Length: 79, dtype: object

In [58]:
learn.predict(new_data.iloc[5])

(Category 电力故障,
 tensor(2),
 tensor([8.7212e-02, 2.7113e-03, 7.6700e-01, 1.3534e-01, 1.2728e-04, 7.6066e-03]))